In [50]:
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
SUITS = ['♠', '♦', '♣', '♥']

deck = [(rank, suit) for rank in RANKS for suit in SUITS]

In [51]:
import random

def deal(deck, n):
    """
    Deals n random hands from the given deck with 2 cards each and returns the hands and the remaining deck.
    
    Parameters:
    - deck (list): The deck of cards to deal from.
    - n (int): The number of hands to deal.
    
    Returns:
    - hands (list): A list of hands, where each hand is a list of cards.
    - remaining_deck (list): The deck of cards after dealing.
    """
    hands = []
    for _ in range(n):
        hand = random.sample(deck, 2)
        deck = [card for card in deck if card not in hand]
        hands.append(hand)
    
    return hands, deck

# Testing the function with 4 random hands
hands, remaining_deck = deal(deck, 4)
hands


[[('Q', '♥'), ('3', '♣')],
 [('7', '♥'), ('A', '♦')],
 [('4', '♥'), ('K', '♦')],
 [('9', '♣'), ('T', '♥')]]

In [52]:
def board(deck):
    """
    Returns a random board of 5 cards from the given deck and the remaining deck.
    
    Parameters:
    - deck (list): The deck of cards to choose from.
    
    Returns:
    - board_cards (list): A list of 5 randomly chosen cards.
    - remaining_deck (list): The deck of cards after choosing the board cards.
    """
    board_cards = random.sample(deck, 5)
    remaining_deck = [card for card in deck if card not in board_cards]
    
    return board_cards, remaining_deck

# Testing the function with the previous remaining deck
board_cards, updated_deck = board(remaining_deck)
board_cards


[('6', '♦'), ('A', '♥'), ('J', '♦'), ('2', '♥'), ('9', '♥')]

In [53]:
from collections import Counter

def hand_strength(hand):
    """
    Evaluates the strength of a hand.
    Returns a tuple (ranking, high_cards) where:
    - ranking is an integer from 1 to 10 (Royal Flush to High Card).
    - high_cards is a list of cards used to break ties.
    """
    # Pre-process the cards
    values = [card[0] for card in hand]
    suits = [card[1] for card in hand]
    value_counts = Counter(values)
    suit_counts = Counter(suits)
    sorted_values = sorted(value_counts, key=lambda x: (value_counts[x], RANKS.index(x)), reverse=True)
    
    # Check for flushes and straights
    flush = any(count >= 5 for count in suit_counts.values())
    consecutive = 0
    prev_rank = None
    for rank in RANKS[::-1]:  # Start from Ace and go down
        if rank in values:
            if prev_rank and RANKS.index(rank) == RANKS.index(prev_rank) - 1:
                consecutive += 1
            else:
                consecutive = 1
            prev_rank = rank
            if consecutive == 5:
                break
    
    straight = consecutive >= 5
    if straight and 'A' in values and '2' in values and '3' in values and '4' in values and '5' in values:
        straight = True
        sorted_values = ['5', '4', '3', '2', 'A']
    
    # Royal Flush
    if flush and straight and sorted_values == ['A', 'K', 'Q', 'J', 'T']:
        return (1, sorted_values)
    # Straight Flush
    if flush and straight:
        return (2, sorted_values[:5])
    # Four of a Kind
    if 4 in value_counts.values():
        quad = [val for val, count in value_counts.items() if count == 4][0]
        return (3, [quad] + [v for v in sorted_values if v != quad])
    # Full House
    if 3 in value_counts.values() and 2 in value_counts.values():
        trips = [val for val, count in value_counts.items() if count == 3][0]
        pair = [val for val, count in value_counts.items() if count == 2][0]
        return (4, [trips, pair])
    # Flush
    if flush:
        flush_suit = [suit for suit, count in suit_counts.items() if count >= 5][0]
        flush_values = sorted([card[0] for card in hand if card[1] == flush_suit], key=lambda x: RANKS.index(x), reverse=True)
        return (5, flush_values[:5])
    # Straight
    if straight:
        return (6, sorted_values[:5])
    # Three of a Kind
    if 3 in value_counts.values():
        trips = [val for val, count in value_counts.items() if count == 3][0]
        return (7, [trips] + [v for v in sorted_values if v != trips][:2])
    # Two Pair
    if list(value_counts.values()).count(2) >= 2:
        pairs = [val for val, count in value_counts.items() if count == 2]
        return (8, pairs + [v for v in sorted_values if v not in pairs][:1])
    # One Pair
    if 2 in value_counts.values():
        pair = [val for val, count in value_counts.items() if count == 2][0]
        return (9, [pair] + [v for v in sorted_values if v != pair][:3])
    # High Card
    return (10, sorted_values[:5])


def winning_hands(hands, board_cards):
    """
    Determine the winning hand(s) from a list of hands given the board cards.
    """
    best_rank = 11  # Start with a value outside of the possible ranks
    winners = []
    best_high_cards = None
    
    for hand in hands:
        combined_cards = hand + board_cards
        rank, high_cards = hand_strength(combined_cards)
        if rank < best_rank:
            best_rank = rank
            winners = [hand]
            best_high_cards = high_cards
        elif rank == best_rank:
            # Tiebreaker based on high cards
            for i in range(5):
                if RANKS.index(high_cards[i]) > RANKS.index(best_high_cards[i]):
                    winners = [hand]
                    best_high_cards = high_cards
                    break
                elif RANKS.index(high_cards[i]) < RANKS.index(best_high_cards[i]):
                    break
            else:
                winners.append(hand)
    
    return winners

# Testing with a sample scenario
sample_hands = [[('A', '♠'), ('K', '♠')], [('Q', '♠'), ('J', '♠')]]
sample_board = [('T', '♠'), ('9', '♠'), ('8', '♠'), ('7', '♠'), ('6', '♠')]

winning_hands(sample_hands, sample_board)


[[('A', '♠'), ('K', '♠')]]

In [54]:
def hand_description(rank, high_cards):
    """
    Returns a description of the hand based on its rank and high cards.
    """
    descriptions = {
        1: "Royal Flush",
        2: "Straight Flush ending in {}".format(high_cards[0]),
        3: "Four of a Kind (Quads) of {}".format(high_cards[0]),
        4: "Full House with {} over {}".format(high_cards[0], high_cards[1]),
        5: "Flush with high card {}".format(high_cards[0]),
        6: "Straight ending in {}".format(high_cards[0]),
        7: "Three of a Kind (Trips) of {}".format(high_cards[0]),
        8: "Two Pair - {} and {}".format(high_cards[0], high_cards[1]),
        9: "One Pair of {}".format(high_cards[0]),
        10: "High Card of {}".format(high_cards[0])
    }
    return descriptions[rank]

def winning_hands(hands, board_cards):
    """
    Determine the winning hand(s) from a list of hands given the board cards.
    Also returns the reason for the win.
    """
    best_rank = 11  # Start with a value outside of the possible ranks
    winners = []
    best_high_cards = None
    
    for hand in hands:
        combined_cards = hand + board_cards
        rank, high_cards = hand_strength(combined_cards)
        if rank < best_rank:
            best_rank = rank
            winners = [hand]
            best_high_cards = high_cards
        elif rank == best_rank:
            # Tiebreaker based on high cards
            for i in range(5):
                if RANKS.index(high_cards[i]) > RANKS.index(best_high_cards[i]):
                    winners = [hand]
                    best_high_cards = high_cards
                    break
                elif RANKS.index(high_cards[i]) < RANKS.index(best_high_cards[i]):
                    break
            else:
                winners.append(hand)
    
    return winners, hand_description(best_rank, best_high_cards)

# Testing with the previous sample scenario
winning_hands(hands, board_cards)


([[('7', '♥'), ('A', '♦')]], 'One Pair of A')

In [55]:
board_cards

[('6', '♦'), ('A', '♥'), ('J', '♦'), ('2', '♥'), ('9', '♥')]

In [56]:
hands

[[('Q', '♥'), ('3', '♣')],
 [('7', '♥'), ('A', '♦')],
 [('4', '♥'), ('K', '♦')],
 [('9', '♣'), ('T', '♥')]]